# Project of text mining

## About this code
The purpose of this notebook is to show an example of how we can extract information from web pages with some level of accuracy.
I've made this notebook as an example to help you in case you needed to scrape infromation in the future.
Here, we are collecting the speeches of previous presidents of the united states

In [8]:
import os
from os import walk
from bs4 import BeautifulSoup
import requests

### Files structures
The folders exist each named by the name of some of the american presidents
inside each folder, there is an html file "initial.txt" scraped from millercenter.org
containing links for all the speeches of each corresponding president

In [9]:
# parent folder
path = 'D:\Efrei\M1\Text mining\TP project1\presidents-speeches/'

# will contain all the data "president name" : ["speeches links"]
presidential_dictionary_list = []
for (root, dirs, files) in walk(path):
    # go through child folders, skip the first parent folder
    if len(dirs) == 1:
        president_speaches_dict = {}
        # get the name of the president from the file path
        president_name = root.split("/")[1]
        president_speaches_dict['president_name'] = president_name
        speaches_links = []
        file_path = "{}/{}".format(root,files[0])
        # open the "initial.txt" file for each president
        with open(file_path) as html_links_file:
            # needed for mac users, otherwise .DS_Store files will get into the way
            if not file_path.endswith('DS_Store'):
                soup = BeautifulSoup(html_links_file)
                for link in soup.findAll('a'):
                    actual_link = link.get('href')
                    # Check for None values and wrong links hidden in a tags
                    if actual_link is not None and actual_link.startswith('/the-presidency'):
                        # extract the speeches links from the <a> tags
                        speaches_links.append(actual_link)
        president_speaches_dict['speeches_links'] = speaches_links
        presidential_dictionary_list.append(president_speaches_dict)

In [10]:
presidential_dictionary_list

[{'president_name': 'Andrew Jackson',
  'speeches_links': ['/the-presidency/presidential-speeches/march-4-1837-farewell-address',
   '/the-presidency/presidential-speeches/december-21-1836-statement-independence-texas',
   '/the-presidency/presidential-speeches/december-5-1836-eighth-annual-message-congress',
   '/the-presidency/presidential-speeches/december-7-1835-seventh-annual-address-congress',
   '/the-presidency/presidential-speeches/december-1-1834-sixth-annual-message-congress',
   '/the-presidency/presidential-speeches/april-21-1834-addendum-protest-senate-censure',
   '/the-presidency/presidential-speeches/april-15-1834-protest-senate-censure',
   '/the-presidency/presidential-speeches/december-12-1833-message-constitutional-rights-and',
   '/the-presidency/presidential-speeches/december-3-1833-fifth-annual-message-congress',
   '/the-presidency/presidential-speeches/september-18-1833-message-regarding-bank-united-states',
   '/the-presidency/presidential-speeches/march-4-18

### Fetch speeches from the site
now that we have the links of the speeches, we can fetch them and extract the content
we will save them in files titled by date__title, corresponding to every president

In [ ]:
# format speech titles to be appropriate for usage as file names
import re
def file_name_format(text):
    return re.sub('[^A-Za-z0-9\-\_]+', '', text)

In [ ]:
for presidential_dictionary in presidential_dictionary_list:
    print("extracting speeches of president {}".format(presidential_dictionary['president_name']))
    speech_counter = 1
    total_speeches = len(presidential_dictionary['speeches_links'])
    for speech_link in presidential_dictionary['speeches_links']:
        try:
            resp = requests.get("https://millercenter.org/{}".format(speech_link))
        except requests.exceptions.RequestException as e:
            print(e)
        print("speech {}/{}".format(speech_counter,total_speeches))
        speech_counter+=1
        if resp.status_code == 200:
            soup = BeautifulSoup(resp._content)
            president_speech = ''
            # the speech is present inside a <div> with class "view-transcript", as a set of <p> tags
            for transcripts in soup.find_all('div', { "class" : "view-transcript"}): # there is only one view-transcript per page
                ps = transcripts.find_all('p')
                president_speech = ' '.join([p.text for p in ps])
            # get the title and date of every speech, and name the file accordingly
            for title in soup.find_all('h2', { "class" : "presidential-speeches--title"}):
                ttl = title.find('span').text
                date_speech_title = ttl.split(":")
                date = date_speech_title[0].strip()
                speech_title = date_speech_title[1].strip()
            with open(path+presidential_dictionary['president_name']+"/speeches/"+date+"__"+file_name_format(speech_title)+".txt","w+") as out_file:
                out_file.write(str(president_speech.encode('utf8')))

# fin
After running this code, all the speeches are be saved inside their corresponding president folder

## Problem noticed

Like we told you, we have a problem, when you will download this file you'll maybe face an isssue, indeed plots that we have done are maybe not showing. To see them please load the notebook and to save time don't launch the deep learning part (if you want to launch it you'll need to download the GloVe file and others dependencies).

In [111]:
import numpy as np
from numpy import empty
from numpy import array
from numpy import asarray
from numpy import zeros
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import CheckButtons
import string
from os import walk
from bs4 import BeautifulSoup
import plotly
import plotly.offline as py
import plotly.graph_objs as go
#import plotly.express as px
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import warnings
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics  import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV 
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix 
from datetime import datetime

warnings.filterwarnings('ignore')

plotly.offline.init_notebook_mode(connected=True)

In [112]:
df = pd.DataFrame(columns=['Name', 'Title', 'Text'])
# List all subdirectories using os.listdir
basepath = path
for entry in os.listdir(basepath):
    if os.path.isdir(os.path.join(basepath, entry)):
        for entry1 in os.listdir(basepath+""+entry):
            if os.path.isdir(os.path.join(basepath+""+entry, entry1)):                
                f = []
                for (dirpath, dirnames, filenames) in walk(basepath+""+entry+"/"+entry1):  
                    f.extend(filenames)
                    for i in filenames:
                        file1 = open(basepath+""+entry+"/"+entry1+"/"+i,"r") 
                        df = df.append({'Name': entry, 'Title' : i, 'Text' : file1.read()}, ignore_index=True)
                        file1.close()
                    break

In [113]:
df

,Name,Title,Text
0,Andrew Jackson,"April 15, 1834__ProtestofSenateCensure.txt",b'To the Senate of the United States:\nIt appe...
1,Andrew Jackson,"April 21, 1834__AddendumtotheProtestofSenateCe...",b'To the Senate of the United States:\nHaving ...
2,Andrew Jackson,"December 1, 1834__SixthAnnualMessagetoCongress...",b'Fellow Citizens of the Senate and House of R...
3,Andrew Jackson,"December 10, 1832__NullificationProclamation.txt","b'By Andrew Jackson, President of the United S..."
4,Andrew Jackson,"December 12, 1833__MessageontheConstitutionalR...",b'I have attentively considered the resolution...
...,...,...,...
370,Thomas Jefferson,"November 3, 1802__AddresstoBrotherHandsomeLake...","b""TO BROTHER HANDSOME LAKE: I have r..."
371,Thomas Jefferson,"November 8, 1804__FourthAnnualMessage.txt",b'To the Senate and House of Representatives o...
372,Thomas Jefferson,"November 8, 1808__EighthAnnualMessage.txt","b"" TO THE SENATE AND HOUSE OF REPRESENTATIVES..."
373,Thomas Jefferson,"October 17, 1803__ThirdAnnualMessage.txt",b'TO THE SENATE AND HOUSE OF REPRESENTATIVES O...


## Preprocessing

In [114]:
df['Text'] = df['Text'].str.lower()

In [115]:
df['Text'] = df['Text'].str.replace("b'", "")
df['Text'] = df['Text'].str.replace('b"', '')
df['Text'] = df['Text'].str.replace('\xc2', '')
df['Text'] = df['Text'].str.replace('\xa0', '')
df['Text'] = df['Text'].str.replace('\xe2', '')
df['Text'] = df['Text'].str.replace('\x99', '')
df['Text'] = df['Text'].str.replace('\x80', '')
df['Text'] = df['Text'].str.replace('\x93', '')
df['Text'] = df['Text'].str.replace('\x99re', '')
df['Text'] = df['Text'].str.replace('\x94', '')
df['Text'] = df['Text'].str.replace('\x99ve', '')
df['Text'] = df['Text'].str.replace('\x9d', '')
df['Text'] = df['Text'].str.replace("'", "")
df['Text'] = df['Text'].str.replace("/", "")
df = df.replace(r'\\n',' ', regex=True) 


df['Cleaned'] = df['Text'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
df

,Name,Title,Text,Cleaned
0,Andrew Jackson,"April 15, 1834__ProtestofSenateCensure.txt",to the senate of the united states: it appears...,to the senate of the united states it appears ...
1,Andrew Jackson,"April 21, 1834__AddendumtotheProtestofSenateCe...",to the senate of the united states: having rea...,to the senate of the united states having reas...
2,Andrew Jackson,"December 1, 1834__SixthAnnualMessagetoCongress...",fellow citizens of the senate and house of rep...,fellow citizens of the senate and house of rep...
3,Andrew Jackson,"December 10, 1832__NullificationProclamation.txt","by andrew jackson, president of the united sta...",by andrew jackson president of the united stat...
4,Andrew Jackson,"December 12, 1833__MessageontheConstitutionalR...",i have attentively considered the resolution o...,i have attentively considered the resolution o...
...,...,...,...,...
370,Thomas Jefferson,"November 3, 1802__AddresstoBrotherHandsomeLake...",to brother handsome lake: i have rec...,to brother handsome lake i have rece...
371,Thomas Jefferson,"November 8, 1804__FourthAnnualMessage.txt",to the senate and house of representatives of ...,to the senate and house of representatives of ...
372,Thomas Jefferson,"November 8, 1808__EighthAnnualMessage.txt",to the senate and house of representatives o...,to the senate and house of representatives o...
373,Thomas Jefferson,"October 17, 1803__ThirdAnnualMessage.txt",to the senate and house of representatives of ...,to the senate and house of representatives of ...


In [116]:
stop = stopwords.words('english')
df['Cleaned'] = df['Cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [117]:
stemmer = PorterStemmer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

def scrub_words(text):
    """Basic cleaning of texts."""

    # remove html markup
    text=re.sub("(<.*?>)","",text)

    #remove non-ascii and digits
    text=re.sub("(\W|\d)"," ",text)

    #remove whitespace
    text=text.strip()
    return text



def test(text):
    return [scrub_words(w) for w in w_tokenizer.tokenize(text)]


def lemmatize_text(text):
    return [stemmer.stem(w) for w in w_tokenizer.tokenize(text)]

df['Cleaned'] = df.Cleaned.apply(lemmatize_text)
df['Cleaned'] = [" ".join(word) for word in df['Cleaned'].values]
df['Cleaned'] = df.Cleaned.apply(test)
df['Cleaned'] = [" ".join(word) for word in df['Cleaned'].values]
df

,Name,Title,Text,Cleaned
0,Andrew Jackson,"April 15, 1834__ProtestofSenateCensure.txt",to the senate of the united states: it appears...,senat unit state appear publish journal senat ...
1,Andrew Jackson,"April 21, 1834__AddendumtotheProtestofSenateCe...",to the senate of the united states: having rea...,senat unit state reason believ certain passag ...
2,Andrew Jackson,"December 1, 1834__SixthAnnualMessagetoCongress...",fellow citizens of the senate and house of rep...,fellow citizen senat hous repres perform duti ...
3,Andrew Jackson,"December 10, 1832__NullificationProclamation.txt","by andrew jackson, president of the united sta...",andrew jackson presid unit state wherea conven...
4,Andrew Jackson,"December 12, 1833__MessageontheConstitutionalR...",i have attentively considered the resolution o...,attent consid resolut senat th instant request...
...,...,...,...,...
370,Thomas Jefferson,"November 3, 1802__AddresstoBrotherHandsomeLake...",to brother handsome lake: i have rec...,brother handsom lake receiv messag write sent ...
371,Thomas Jefferson,"November 8, 1804__FourthAnnualMessage.txt",to the senate and house of representatives of ...,senat hous repres unit state peopl fellowcitiz...
372,Thomas Jefferson,"November 8, 1808__EighthAnnualMessage.txt",to the senate and house of representatives o...,senat hous repres unit state would sourc fello...
373,Thomas Jefferson,"October 17, 1803__ThirdAnnualMessage.txt",to the senate and house of representatives of ...,senat hous repres unit state call togeth fello...


**We split the dataset into 80/20 for every president set**

In [118]:
X_trainFinal = pd.Series([])
X_testFinal = pd.Series([])
y_trainFinal = pd.Series([])
y_testFinal = pd.Series([])

X = df.iloc[:,3]
y = df.iloc[:,0]

for i in df.Name.unique():
    df2 = df.groupby("Name")
    df3 = df2.get_group(i)
    X_train, X_test, y_train, y_test = train_test_split(df3["Cleaned"], df3["Name"], test_size=0.2, random_state=21)
    X_trainFinal = X_trainFinal.append(X_train)
    X_testFinal = X_testFinal.append(X_test)
    y_trainFinal = y_trainFinal.append(y_train)
    y_testFinal = y_testFinal.append(y_test)

**TF-IDF**

In [119]:
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_trainFinal)
test_vectors = vectorizer.transform(X_testFinal)

**KNN classification**

In [120]:
knn = KNeighborsClassifier()
knn.fit(train_vectors,y_trainFinal)
pred = knn.predict(test_vectors)
accuracy_score(pred,y_testFinal)

0.759493670886076

**Logistic regression classification**

In [121]:
clf = LogisticRegression(random_state=0).fit(train_vectors, y_trainFinal)
pred = clf.predict(test_vectors)
accuracy_score(pred,y_testFinal)

0.7974683544303798

**We do a GridSearchCV for our LinearSVC model with different regularization parameters**

In [122]:
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000]}  
  
grid = GridSearchCV(svm.LinearSVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search
grid.fit(train_vectors, y_trainFinal)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] C=0.1 ...........................................................
[CV] ............................... C=0.1, score=0.641, total=   0.1s
[CV] C=0.1 ...........................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ............................... C=0.1, score=0.724, total=   0.1s
[CV] C=0.1 ...........................................................
[CV] ............................... C=0.1, score=0.747, total=   0.1s
[CV] C=1 .............................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV] ................................. C=1, score=0.738, total=   0.2s
[CV] C=1 .............................................................
[CV] ................................. C=1, score=0.827, total=   0.2s
[CV] C=1 .............................................................
[CV] ................................. C=1, score=0.863, total=   0.3s
[CV] C=10 ............................................................
[CV] ................................ C=10, score=0.767, total=   0.5s
[CV] C=10 ............................................................
[CV] ................................ C=10, score=0.867, total=   0.4s
[CV] C=10 ............................................................
[CV] ................................ C=10, score=0.884, total=   0.5s
[CV] C=100 ...........................................................
[CV] ............................... C=100, score=0.767, total=   1.0s
[CV] C=100 ...........................................................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    7.7s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='warn', n_jobs=None, param_grid={'C': [0.1, 1, 10, 100, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

**LinearSVC classification**

In [123]:
cf = svm.LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0)
cf.fit(train_vectors, y_trainFinal)
pred = cf.predict(test_vectors)
print(accuracy_score(pred,y_testFinal))

print(classification_report(y_testFinal, pred)) 

0.9113924050632911
                       precision    recall  f1-score   support

       Andrew Jackson       0.86      1.00      0.92         6
         Barack Obama       0.83      1.00      0.91        10
         Bill Clinton       0.78      0.88      0.82         8
         Donald Trump       1.00      0.80      0.89         5
Franklin D. Roosevelt       1.00      1.00      1.00        10
    George H. W. Bush       1.00      0.40      0.57         5
       George W. Bush       1.00      0.88      0.93         8
    George Washington       1.00      0.80      0.89         5
     Richard M. Nixon       0.83      1.00      0.91         5
        Ronald Reagan       0.92      1.00      0.96        12
     Thomas Jefferson       1.00      1.00      1.00         5

             accuracy                           0.91        79
            macro avg       0.93      0.89      0.89        79
         weighted avg       0.92      0.91      0.91        79



### Deep learning model

In [124]:
dfDummies = pd.get_dummies(df['Name'])

name_label = dfDummies[["Andrew Jackson", "Barack Obama", "Bill Clinton", "Franklin D. Roosevelt", "George W. Bush", "George Washington", "Richard M. Nixon", "Ronald Reagan", "Thomas Jefferson"]]
name_label.head()

,Andrew Jackson,Barack Obama,Bill Clinton,Franklin D. Roosevelt,George W. Bush,George Washington,Richard M. Nixon,Ronald Reagan,Thomas Jefferson
0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0


**X and y**

In [125]:
X = []
sentences = list(df["Cleaned"])

for sen in sentences:
    X.append(sen)

y = name_label.values

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

**Tokenization**

In [127]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1
maxlen = 3000

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

**We use a pretrained GloVe to do the word embedding**

To download GloVe : https://nlp.stanford.edu/projects/glove/ | We took the Wikipedia2014 + Gigaword

We used -> glove.6B.100d

In [128]:
embeddings_dictionary = dict()

glove_file = open('D:/Efrei/M1/Text mining/TP project1/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

**We create the LSTM model with 1 input layer | Embedding layer | LSTM layer | Dense layer**

In [129]:
deep_inputs = Input(shape=(maxlen,))

embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(9, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [130]:
print(model.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3000)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 3000, 100)         1906700   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 1161      
Total params: 2,025,109
Trainable params: 118,409
Non-trainable params: 1,906,700
_________________________________________________________________
None


**We tried with many epoch (1, 3, 5, 10) but it is not very usefull to add more than 3**

In [131]:
history = model.fit(X_train, y_train, batch_size=128, epochs=3, verbose=1, validation_split=0.2)

Train on 240 samples, validate on 60 samples
Epoch 1/3
240/240 [==============================] - 51s 215ms/step - loss: 0.6896 - acc: 0.8653 - val_loss: 0.6756 - val_acc: 0.8981
Epoch 2/3
240/240 [==============================] - 31s 129ms/step - loss: 0.6746 - acc: 0.8995 - val_loss: 0.6588 - val_acc: 0.9074
Epoch 3/3
240/240 [==============================] - 23s 94ms/step - loss: 0.6592 - acc: 0.9032 - val_loss: 0.6379 - val_acc: 0.9037


In [132]:
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

75/75 [==============================] - 2s 33ms/step
Test Score: 0.6220561830202739
Test Accuracy: 0.9007407426834106


# Sentiment Analysis

### Order the presidents in chronological order, from oldest to newest

In [133]:
df["Chrono"] = ""
for i, j in df.iterrows(): 
    if j.Name == "George Washington":
        df["Chrono"][i] = 1
    if j.Name == "Thomas Jefferson":
        df["Chrono"][i] = 2
    if j.Name == "Andrew Jackson":
        df["Chrono"][i] = 3
    if j.Name == "Franklin D. Roosevelt":
        df["Chrono"][i] = 4
    if j.Name == "Richard M. Nixon":
        df["Chrono"][i] = 5
    if j.Name == "Ronald Reagan":
        df["Chrono"][i] = 6
    if j.Name == "George H. W. Bush":
        df["Chrono"][i] = 7
    if j.Name == "Bill Clinton":
        df["Chrono"][i] = 8
    if j.Name == "George W. Bush":
        df["Chrono"][i] = 9
    if j.Name == "Barack Obama":
        df["Chrono"][i] = 10
    if j.Name == "Donald Trump":
        df["Chrono"][i] = 11

### Order each of their speeches in chronological order as well, from oldest to newest.

In [134]:
df["Date"] = ""

def extractMax(input): 
    numbers = re.findall('\d+',input) 
    numbers = map(int,numbers) 
    return max(numbers)  
     
for i, j in df.iterrows(): 
    a = j.Title.split(",")
    res = extractMax(str(a[1]))
    a[1] = str(res)
    a = ' '.join(a)

    d = datetime.strptime(a, '%B %d %Y')
    d = d.strftime('%Y-%m-%d')
    df["Date"][i] = d
    
df['Date']= pd.to_datetime(df['Date'])

In [135]:
df.sort_values("Chrono", axis = 0, ascending = True, 
                 inplace = True, na_position ='last') 
df.sort_values("Date", axis = 0, ascending = True, 
                 inplace = True, na_position ='last') 
df

,Name,Title,Text,Cleaned,Chrono,Date
249,George Washington,"April 30, 1789__FirstInauguralAddress.txt",fellow citizens of the senate and the house of...,fellow citizen senat hous repres among vicissi...,1,1789-04-30
266,George Washington,"October 3, 1789__ThanksgivingProclamation.txt",whereas it is the duty of all nations to ackno...,wherea duti nation acknowledg provid almighti ...,1,1789-10-03
259,George Washington,"January 8, 1790__FirstAnnualMessagetoCongress.txt",fellow citizens of the senate and house of rep...,fellow citizen senat hous repres embrac great ...,1,1790-01-08
257,George Washington,"December 8, 1790__SecondAnnualMessagetoCongres...",fellow citizens of the senate and house of re...,fellow citizen senat hous repres meet feel muc...,1,1790-12-08
254,George Washington,"December 29, 1790__TalktotheChiefsandCounselor...","i the president of the united states, by my o...",presid unit state mouth written speech sign ha...,1,1790-12-29
...,...,...,...,...,...,...
134,Donald Trump,"September 24, 2019__RemarksattheUnitedNationsG...",president trump:\xc2\xa0 thank you very much.\...,presid trumpxc xa thank muchxc xa mr presid mr...,11,2019-09-24
136,Donald Trump,"September 25, 2019__PressConference.txt",president trump:\xc2\xa0 thank you very much.\...,presid trumpxc xa thank muchxc xa thank well t...,11,2019-09-25
132,Donald Trump,"October 27, 2019__StatementonthetheDeathofAbuB...","last night, the united states brought the worl...",last night unit state brought worldxe x x num...,11,2019-10-27
125,Donald Trump,"January 3, 2020__RemarksontheKillingofQasemSol...","hello, everybody.\xc2\xa0well, thank you very ...",hello everybodyxc xa wel thank muchxc xa and g...,11,2020-01-03


### VADER sentiment analyzer on the processed speeches and extract the sentiment of every speech.

In [136]:
def sentiment_scores(sentence): 
  
    # Create a SentimentIntensityAnalyzer object. 
    sid_obj = SentimentIntensityAnalyzer() 
  
    # polarity_scores method of SentimentIntensityAnalyzer 
    # oject gives a sentiment dictionary. 
    # which contains pos, neg, neu, and compound scores. 
    sentiment_dict = sid_obj.polarity_scores(sentence) 
  
    # decide sentiment as positive, negative and neutral 
    if sentiment_dict['compound'] >= 0.05 : 
        return 0,sentiment_dict['neg'],sentiment_dict['pos']
  
    elif sentiment_dict['compound'] <= - 0.05 : 
        return 1,sentiment_dict['neg'],sentiment_dict['pos']
df["Sentiment Speech"] = ""
df["Ratio positive"] = ""
df["Ratio negative"] = ""

for k, j in df.iterrows():
    sentiment,ratioN,ratioP = sentiment_scores(j.Cleaned)
    if(sentiment == 0):
        df["Sentiment Speech"][k] = "Positive"
        df["Ratio positive"][k] = ratioP
        df["Ratio negative"][k] = ratioN
    elif(sentiment == 1):
        df["Sentiment Speech"][k] = "Negative"
        df["Ratio positive"][k] = ratioP
        df["Ratio negative"][k] = ratioN
df

,Name,Title,Text,Cleaned,Chrono,Date,Sentiment Speech,Ratio positive,Ratio negative
249,George Washington,"April 30, 1789__FirstInauguralAddress.txt",fellow citizens of the senate and the house of...,fellow citizen senat hous repres among vicissi...,1,1789-04-30,Positive,0.264,0.037
266,George Washington,"October 3, 1789__ThanksgivingProclamation.txt",whereas it is the duty of all nations to ackno...,wherea duti nation acknowledg provid almighti ...,1,1789-10-03,Positive,0.364,0.014
259,George Washington,"January 8, 1790__FirstAnnualMessagetoCongress.txt",fellow citizens of the senate and house of rep...,fellow citizen senat hous repres embrac great ...,1,1790-01-08,Positive,0.257,0.033
257,George Washington,"December 8, 1790__SecondAnnualMessagetoCongres...",fellow citizens of the senate and house of re...,fellow citizen senat hous repres meet feel muc...,1,1790-12-08,Positive,0.202,0.077
254,George Washington,"December 29, 1790__TalktotheChiefsandCounselor...","i the president of the united states, by my o...",presid unit state mouth written speech sign ha...,1,1790-12-29,Positive,0.171,0.121
...,...,...,...,...,...,...,...,...,...
134,Donald Trump,"September 24, 2019__RemarksattheUnitedNationsG...",president trump:\xc2\xa0 thank you very much.\...,presid trumpxc xa thank muchxc xa mr presid mr...,11,2019-09-24,Positive,0.194,0.083
136,Donald Trump,"September 25, 2019__PressConference.txt",president trump:\xc2\xa0 thank you very much.\...,presid trumpxc xa thank muchxc xa thank well t...,11,2019-09-25,Positive,0.109,0.042
132,Donald Trump,"October 27, 2019__StatementonthetheDeathofAbuB...","last night, the united states brought the worl...",last night unit state brought worldxe x x num...,11,2019-10-27,Negative,0.16,0.254
125,Donald Trump,"January 3, 2020__RemarksontheKillingofQasemSol...","hello, everybody.\xc2\xa0well, thank you very ...",hello everybodyxc xa wel thank muchxc xa and g...,11,2020-01-03,Negative,0.188,0.226


In [137]:
df

,Name,Title,Text,Cleaned,Chrono,Date,Sentiment Speech,Ratio positive,Ratio negative
249,George Washington,"April 30, 1789__FirstInauguralAddress.txt",fellow citizens of the senate and the house of...,fellow citizen senat hous repres among vicissi...,1,1789-04-30,Positive,0.264,0.037
266,George Washington,"October 3, 1789__ThanksgivingProclamation.txt",whereas it is the duty of all nations to ackno...,wherea duti nation acknowledg provid almighti ...,1,1789-10-03,Positive,0.364,0.014
259,George Washington,"January 8, 1790__FirstAnnualMessagetoCongress.txt",fellow citizens of the senate and house of rep...,fellow citizen senat hous repres embrac great ...,1,1790-01-08,Positive,0.257,0.033
257,George Washington,"December 8, 1790__SecondAnnualMessagetoCongres...",fellow citizens of the senate and house of re...,fellow citizen senat hous repres meet feel muc...,1,1790-12-08,Positive,0.202,0.077
254,George Washington,"December 29, 1790__TalktotheChiefsandCounselor...","i the president of the united states, by my o...",presid unit state mouth written speech sign ha...,1,1790-12-29,Positive,0.171,0.121
...,...,...,...,...,...,...,...,...,...
134,Donald Trump,"September 24, 2019__RemarksattheUnitedNationsG...",president trump:\xc2\xa0 thank you very much.\...,presid trumpxc xa thank muchxc xa mr presid mr...,11,2019-09-24,Positive,0.194,0.083
136,Donald Trump,"September 25, 2019__PressConference.txt",president trump:\xc2\xa0 thank you very much.\...,presid trumpxc xa thank muchxc xa thank well t...,11,2019-09-25,Positive,0.109,0.042
132,Donald Trump,"October 27, 2019__StatementonthetheDeathofAbuB...","last night, the united states brought the worl...",last night unit state brought worldxe x x num...,11,2019-10-27,Negative,0.16,0.254
125,Donald Trump,"January 3, 2020__RemarksontheKillingofQasemSol...","hello, everybody.\xc2\xa0well, thank you very ...",hello everybodyxc xa wel thank muchxc xa and g...,11,2020-01-03,Negative,0.188,0.226


In [138]:
donald_trump = pd.DataFrame()
donald_trump = df.where(df['Name'] == "Donald Trump")
donald_trump = donald_trump[donald_trump['Name'].notnull()]

**Usefull function to creathe the graph**

In [139]:
names = df['Name'].unique()

def createData(name, df):
    """
    name : It is the list of names inside the dataframe
    df : It is the dataframe which contains the informations
    
    return a list of all plots with the corresponding data of the president
    """
    l = []
    for name in df['Name'].unique():
        tempo = pd.DataFrame()
        tempo = df.where(df['Name'] == name)
        tempo = tempo[tempo['Name'].notnull()]
        #Donald trump
        neg = go.Scatter(x=tempo['Date'],
                        y=tempo['Ratio negative'],
                        name='Ratio negative',
                        line=dict(color='red'),
                        line_shape='spline')

        pos = go.Scatter(x=tempo['Date'],
                        y=tempo['Ratio positive'],
                        name='Ratio positive',
                        line=dict(color='green'),
                            line_shape='spline')
        l.append(neg)
        l.append(pos)
        
    return l

l = createData(names, df)

In [140]:
data = l

def buttonsUpdateMenu(names):
    """
    names : Names of presidents present in the dataframe
    
    return a list usefull to the graph (control the visibility and buttons of the graph)
    """
    i = 0
    l = []
    visibleList = [False, False, False, False, False,False, False,False, False,False, False,False, False,False, False,False, False,False, False,False, False,False]
    for name in names:
        visibleList[i] = True
        i = i+1
        visibleList[i] = True
        i = i+1
        l.append(dict(label = name,
             method = 'update',
             args = [{'visible': visibleList},
                     {'title': name}]))
        visibleList = [False, False, False, False, False,False, False,False, False,False, False,False, False,False, False,False, False,False, False,False, False,False]

    return l

updateButtonsList = list([
    dict(type="buttons",
         active=-1,
         buttons= buttonsUpdateMenu(names),
    )
])

**Graph for all presidents (1 by 1)**

In [141]:
updatemenus = updateButtonsList

layout = dict(title='President and Sentiment', showlegend=False,
              updatemenus=updatemenus, xaxis= {
    "side": "bottom", 
    "type": "category", 
    "range": [0, 21], 
    "title": {"text": "Date"}, 
    "domain": [0, 1], 
    "autorange": True, 
    "showspikes": False})

fig = dict(data=l, layout=layout)
py.iplot(fig, filename='update_button')

**Graph for all presidents**

In [142]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=df['Date'], y=df['Ratio positive'],
                    mode='lines+markers',
                    name='Ratio positive'))

fig.add_trace(go.Scatter(x=df['Date'], y=df['Ratio negative'],
                    mode='lines+markers',
                    name='Ratio negative'))

fig.show()

In [143]:
neg = []
pos = []

total = pd.DataFrame()

for i in df.Name.unique():
    df2 = df.groupby("Name")
    df3 = df2.get_group(i)
    a_row = pd.Series([i, df3["Sentiment Speech"].str.count("Negative").sum(),df3["Sentiment Speech"].str.count("Positive").sum()])
    row_df = pd.DataFrame([a_row])
    total = pd.concat([row_df, total], ignore_index=True)
    neg.append(df3["Sentiment Speech"].str.count("Negative").sum())
    pos.append(df3["Sentiment Speech"].str.count("Positive").sum())

In [144]:
total = total.rename(columns={0: "Name", 1: "Total Negative", 2: "Total Positive"})
total = total.iloc[::-1]
total

,Name,Total Negative,Total Positive
10,George Washington,1,20
9,Thomas Jefferson,1,23
8,Andrew Jackson,0,26
7,Franklin D. Roosevelt,13,36
6,Richard M. Nixon,3,20
5,Ronald Reagan,3,56
4,George H. W. Bush,3,20
3,Bill Clinton,4,35
2,George W. Bush,9,30
1,Barack Obama,6,44


In [145]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=total["Name"],y=total["Total Positive"],
    name='Positive',
    marker_color='blue'
))
fig.add_trace(go.Bar(
    x=total["Name"],y=total["Total Negative"],
    name='Negative',
    marker_color='indianred'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()